In [2]:
import json
from apiclient.discovery import build

with open('/Users/kii/work/access_analysis/json/youtube_data.json') as f:
    data = json.load(f)
API_KEY = data['API_KEY']

youtube = build('youtube', 'v3', developerKey=API_KEY)

search_response = youtube.search().list(
part='snippet',
#検索したい文字列を指定
q='動物',
#視聴回数が多い順に取得
order='viewCount',
type='video',
).execute()

In [3]:
import pandas as pd

def get_video_info(part, q, order, type, num):
    dic_list = []
    search_response = youtube.search().list(part=part,q=q,order=order,type=type)
    output = youtube.search().list(part=part,q=q,order=order,type=type).execute()

    #一度に5件しか取得できないため何度も繰り返して実行
    for i in range(num):        
        dic_list = dic_list + output['items']
        search_response = youtube.search().list_next(search_response, output)
        output = search_response.execute()

    df = pd.DataFrame(dic_list)
    #各動画毎に一意のvideoIdを取得
    df1 = pd.DataFrame(list(df['id']))['videoId']
    #各動画毎に一意のvideoIdを取得必要な動画情報だけ取得
    df2 = pd.DataFrame(list(df['snippet']))[['channelTitle','publishedAt','channelId','title','description']]
    ddf = pd.concat([df1,df2], axis = 1)

    return ddf

In [4]:
df = get_video_info(part='snippet',q='動物',order='viewCount',type='video',num = 20)

In [5]:
#videoIdを入力することで、その動画の具体的な再生回数やいいね数を取得する関数を作成
def get_statistics(id):
    statistics = youtube.videos().list(part = 'statistics', id = id).execute()['items'][0]['statistics']
    return statistics

df_static = pd.DataFrame(list(df['videoId'].apply(lambda x : get_statistics(x))))

df_output = pd.concat([df,df_static], axis = 1)

df_output

,videoId,channelTitle,publishedAt,channelId,title,description,viewCount,likeCount,favoriteCount,commentCount
0,cd0Fy92_w_s,ねこチャックの日常,2021-09-21T10:00:16Z,UC-R3YNdUfeQ_LErDhy0atVA,びっくりする猫,,679167876,10055322,0,31708
1,Fn7NLWHJw4s,相信音樂BinMusic,2016-05-20T12:00:01Z,UC7ovu6a8ydIbDy0fAKmoZ9A,MAYDAY五月天 [ 派對動物Party Animal ] Official Music ...,免費訂閱▷相信音樂YouTube官方頻道https://www.youtube.com/us...,83447645,216727,0,9332
2,ontm1vqONVY,アドベンチャーワールド公式,2022-01-06T08:22:30Z,UCVEmpbL5VzfXsULPFeRsj4Q,飼育スタッフ VS エンペラーペンギンの赤ちゃん,じっとしてほしい飼育スタッフとじっとしていられない赤ちゃんの攻防戦…！？ 2021年10月1...,34369137,2078677,0,33673
3,28LbyiJZXTg,PECOチャンネルさん【癒やしのペット動画】,2021-04-05T10:00:00Z,UCdU0lALBi35E1vqwpbZrzSw,妹ちゃんが泣き出しちゃった…😭 その時、ワンコがとった優しい行動✨ #Shorts,シベリアンハスキーのロキくんとバーニーズマウンテンドッグのニコちゃん。 妹ちゃんがニコちゃん...,31603534,446513,0,NaN
4,TuA1lAq8jY4,毎日甘いもの食べたい,2023-04-29T13:20:00Z,UCzGXhNrC633Y_Rr8pdFBx7g,名古屋にある世界一かわいい動物クレープ屋さん🐹ホイップクリームを大量に使ってアルパカの形を作...,アルパカクレープ 880円 ▽撮影店舗▽ クレープス･ド･アンヌ 愛知県名古屋市中村区椿町1...,30460452,908741,0,2014
...,...,...,...,...,...,...,...,...,...,...
95,anf0nVJue1M,かぎあな工房〜架空の生物紹介?〜,2023-01-14T08:43:36Z,UCOpdOBYB4N2MsQ8kZPrszDA,臭すぎる動物3選 #shorts #かぎあな紹介 #サリズミ,,5224484,85915,0,1510
96,EH17rEHKpGQ,旅人夢語,2021-12-05T13:02:09Z,UCj3FBrCXdM0LmLynRBQlaZg,若非是被鏡頭記錄下來，你永遠不會相信動物們的機智竟會恐怖如斯！,就像大家所知道，有些動物很是聰明但若非是鏡頭記錄下來這些畫面相信你永遠也不會知道動物們的機智...,5206995,56986,0,870
97,lVbbIiNOCW4,タカラトミー,2020-05-15T07:00:19Z,UClVgBLXWrw3TV4Wm0d7qrFQ,【公式】火山が大噴火!アニア 動物 恐竜 大混乱！タカラトミーオールスター!ストップモーショ...,どうがをみるときは、へやをあかるくして、めをちかづけすぎないようにしてみてね。 詳しくはこち...,5201766,7294,0,0
98,Lnxc3xc7XjU,東海オンエア,2019-04-05T11:57:45Z,UCutJqz56653xV2wwSvut_hQ,岡崎って動物出まくるから実質ナイトサファリなんじゃね？,ドライブ動画はトークがメインになりがちです。 自然とアカン話も増えがちです。 この動画もすご...,5197083,71322,0,6531


In [6]:
df_output.to_excel('キリン.xlsx')
